我可以幫你修改程式以使用 multilingual-e5-large 模型。這需要使用 transformers 庫而不是 Google Gemini API。
主要的改動包括:

1. 使用 transformers 庫載入 multilingual-e5-large 模型和 tokenizer
2. 實現了一個 DocumentSearch 類來封裝所有功能
3. 支援 query/passage 前綴,這是 E5 模型的特殊要求
4. 使用正確的模型輸出方式(CLS token)和向量正規化

要使用這個代碼,你需要先安裝必要的套件:

```bash
pip install transformers torch pandas numpy
```

與原本使用 Gemini API 的版本相比,主要差異是:

1. 不需要 API key,完全離線運作
2. 使用開源模型,可以自由修改和部署
3. 支援更多語言(multilingual)
4. 向量維度和相似度計算方式略有不同

你可以根據需要調整模型參數,例如:
- 修改 max_length 
- 使用不同的模型變體(例如 e5-base)
- 調整向量正規化方式
- 加入批次處理以提高效率

需要我針對某個特定部分做更詳細的說明嗎?

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd

class DocumentSearch:
    def __init__(self):
        # 初始化模型和tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
        self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        
    def get_embedding(self, text, is_query=False):
        # 添加前缀
        if is_query:
            text = f"query: {text}"
        else:
            text = f"passage: {text}"
            
        # 编码文本
        inputs = self.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        
        # 获取嵌入向量
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :]  # 使用 [CLS] token的输出
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
            
        return embeddings.cpu().numpy()[0]

    def create_document_embeddings(self, df):
        """为文档创建嵌入向量"""
        embeddings = []
        for _, row in df.iterrows():
            # 组合标题和内容
            text = f"Title: {row['Title']}\nText: {row['Text']}"
            embedding = self.get_embedding(text, is_query=False)
            embeddings.append(embedding)
        return embeddings

    def find_best_passage(self, query, df):
        """查找最相关的文档"""
        # 获取查询的嵌入向量
        query_embedding = self.get_embedding(query, is_query=True)
        
        # 计算相似度
        similarities = np.dot(np.stack(df['Embeddings'].values), query_embedding)
        
        # 返回最相关的文档
        best_idx = np.argmax(similarities)
        return df.iloc[best_idx]['Text'], similarities[best_idx]

# 示例使用
if __name__ == "__main__":
    # 初始化文档搜索
    doc_search = DocumentSearch()
    
    # 准备示例数据
    documents = [
        {
            "Title": "操作氣候控制系統",
            "Text": "您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。"
        },
        {
            "Title": "觸控螢幕",
            "Text": "您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。"
        }
    ]
    
    # 创建DataFrame
    df = pd.DataFrame(documents)
    
    # 生成嵌入向量
    df['Embeddings'] = doc_search.create_document_embeddings(df)
    
    # 测试查询
    query = "觸控螢幕？"
    best_text, similarity = doc_search.find_best_passage(query, df)
    
    print(f"Query: {query}")
    print(f"Best matching text (similarity: {similarity:.3f}):")
    print(best_text)
